In [4]:
import pandas as pd
import numpy as np
import time
import requests
import re

In [5]:
# Helper functions
def timer_function(func, data, n_run=1):
    ti = time.time()
    for i in range(n_run):
        func(data)
    tf = time.time()
    
    print(f'Runtime: {(tf - ti)/n_run} s')

def scrape_data(day):
    """
    fetches the input data from the website
    day: the day of december to fetch
    """
    # Fetch session cookie value from local file
    with open("session.txt",'r') as f:
        key = f.read()
    cookie = {'session': key}
    url= f"https://adventofcode.com/2021/day/{day}/input"

    with requests.Session() as s:
        r = s.get(url, cookies = cookie).text
        if 'Please log in' in r:
            print(r)
            input_lst = None
        else:
            # Data always end with a blanc line. Data is split on line change
            input_lst = re.split('\n', r)[:-1]

    return input_lst

# December 1


In [6]:
# data import
input_test = np.array([199,200,208,210,200,207,240,269,260,263])
input_1 = scrape_data(day=1)
input_1 = [int(i) for i in input_1]
print('Number of input rows:', len(input_1))

Number of input rows: 2000


In [7]:
# Part 1
def dec1_part1(data):
    answer = np.sum(np.diff(data) > 0)
    return answer

answer = dec1_part1(input_1)
print(f'Solution december 1. - part 1: ', answer)
timer_function(func=dec1_part1, data=input_1, n_run=1000)

Solution december 1. - part 1:  1583
Runtime: 0.0001420297622680664 s


In [8]:
# Part 2
def dec1_part2(data):
    window = 3
    rolling_sum =[np.sum(data[i:i + window]) for i in range(0, len(data) + 1 - window)]
    answer = dec1_part1(rolling_sum)

    return rolling_sum, answer

rolling_sum, answer = dec1_part2(input_1)
print(f'Solution december 1. - part 2: ', answer)
timer_function(func=dec1_part2,data=input_1, n_run=10)

Solution december 1. - part 2:  1627
Runtime: 0.008501911163330078 s


# December 2


In [9]:
input_test = ['forward 5','down 5','forward 8','up 3','down 8','forward 2']
input_2 = scrape_data(day=2)
print('Number of input rows:', len(input_2))

Number of input rows: 1000


In [10]:
# Solution and test input
def split_direction(df, direction, sign):
    mask = df['raw'].str.contains(direction)
    df.loc[mask, 'direction'] = direction[0]
    df.loc[mask, 'step'] = df.loc[mask, 'raw'].str.replace(direction, '').astype(int)*sign
    
    return df

def dec2_part1(data):
    df_data = pd.DataFrame(data).rename(columns={0:'raw'})
    
    df_data = split_direction(df_data, 'forward ', +1)
    df_data = split_direction(df_data, 'down ', +1)
    df_data = split_direction(df_data, 'up ', -1)
    
    mask_fwd = df_data['direction'] == 'f'
    pos_horizontal = df_data.loc[mask_fwd, 'step'].sum()
    pos_vertical = df_data.loc[~mask_fwd, 'step'].sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df_data, answer


pos_horizontal, pos_vertical, df_input, answer = dec2_part1(input_2)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part1,data=input_2, n_run=10)

Position (horizontal/vertical): 1939.0 , 1109.0
Solution:  2150351.0
Runtime: 0.006401562690734863 s


In [11]:
# December  2 - Part 2
def dec2_part2(df):    
    mask_fwd = df['direction'] == 'f'
    
    df['aim_step'] = 0
    df.loc[~mask_fwd, 'aim_step'] = df.loc[~mask_fwd, 'step']
    df['aim'] = df['aim_step'].cumsum()

    pos_horizontal = df.loc[mask_fwd, 'step'].sum()
    pos_vertical = (df.loc[mask_fwd, 'step']*df.loc[mask_fwd, 'aim']).sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df, answer

pos_horizontal, pos_vertical, df_input, answer = dec2_part2(df_input)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part2,data=df_input, n_run=10)

Position (horizontal/vertical): 1939.0 , 950357.0
Solution:  1842742223.0
Runtime: 0.0018003463745117187 s


# December 3

In [12]:
input_test = ['00100','11110','10110','10111','10101','01111','00111','11100','10000','11001','00010','01010']
input_3 = scrape_data(day=3)
print('Number of input rows:', len(input_3))

Number of input rows: 1000


In [13]:
# Solution part 1 and test
def dec3_part1(data):
    n_digit = len(data[0])
    n_row = len(data)
    
    digit_sum = np.zeros(n_digit)
    
    for row in data:
        row_lst = [int(i) for i in row]
        digit_sum = np.add(digit_sum, row_lst)
    
    mask = digit_sum >= n_row/2
    gamma = ''.join(map(str, 1*(mask)))
    epsilon = ''.join(map(str, 1*(~mask)))
    
    answer = int(gamma,2)*int(epsilon,2)
    
    return gamma, epsilon, answer
    
print(dec3_part1(input_3))
timer_function(func=dec3_part1,data=input_3, n_run=10)

('010001110111', '101110001000', 3374136)
Runtime: 0.004302334785461426 s


In [14]:
def find_rating(rating, bit):
    n_digit = len(rating[0])
    n_row = len(rating)
    idx = 0
    
    while (len(rating) > 1) & (idx < n_digit):
        half_len = len(rating)/2
        digit_sum = 0
        for row in rating:
            digit_sum += int(row[idx])
            
        digit_even = digit_sum == half_len
        
        if digit_even:
            rating = [entry for entry in rating if entry[idx] == bit]
        else:
            if bit == '0':
                digit_match = 1*(digit_sum < half_len)
            else:
                digit_match = 1*(digit_sum > half_len)

            rating = [entry for entry in rating if entry[idx] == str(digit_match)]

        idx += 1
    
    # select the only remaining correct entry if more exist
    rating = [entry for entry in rating if entry[-1] == bit][0]
    
    return rating

def dec3_part2(data):
    o_rating = find_rating(rating=data, bit='1')
    co2_rating = find_rating(rating=data, bit='0')
        
    answer = int(o_rating,2)*int(co2_rating,2)
    
    return o_rating, co2_rating, answer

print(dec3_part2(input_3))
timer_function(func=dec3_part2,data=input_3, n_run=10)

('011101110101', '100100010010', 4432698)
Runtime: 0.001300477981567383 s


# December 4

In [15]:
input_test = [
    '7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1',
    '',
    '22 13 17 11  0',
    '8  2 23  4 24',
    '21  9 14 16  7',
    '6 10  3 18  5',
     '1 12 20 15 19',
    '',
    '3 15  0  2 22',
    '9 18 13 17  5',
    '19  8  7 25 23',
    '20 11 10 24  4',
    '14 21 16 12  6',
    '',
    '14 21 17 24  4',
    '10 16 15  9 19',
    '18  8 23 26 20',
    '22 11 13  6  5',
     '2  0 12  3  7']
input_4 = scrape_data(day=4)

In [16]:
def create_data_structure(data):
    numbers = list(map(int, re.split(',', data[0])))
    idx_start = 2
    idx_end = len(data) - 1
    n_size = 5

    boards = []
    for board_start in range(idx_start, idx_end, n_size + 1):
        board = []
        for idx in range(board_start, board_start + n_size):
            board.append(list(map(int, re.split('\s+', data[idx].strip()))))


        for col in np.transpose(board):
            board.append(col)
            
        boards.append(board)

    board_sets = [[set(row) for row in board] for board in boards]
    return numbers, board_sets

numbers, board_sets = create_data_structure(input_4)

In [17]:
def dec4_part1(data):
    numbers, board_sets = create_data_structure(data)
    
    keep_going = True
    for idx, n in enumerate(numbers):
        for n_board, board in enumerate(board_sets):
            for row in board:
                row.discard(n)

                if len(row) == 0:
                    keep_going = False
                    board_winner = board

        if keep_going:
            continue
        else:
            break
            
            
    remainders = set(i for row in board_winner for i in row)
    winner_board = idx
    answer = sum(remainders)*n
    return n_board, answer

print('Answer:', dec4_part1(input_4))
timer_function(func=dec4_part1,data=input_4, n_run=10)

Answer: (99, 46920)
Runtime: 0.0043819665908813475 s


In [18]:
def dec4_part2(data):
    numbers, board_sets = create_data_structure(data)
    keep_going = True
    for idx, n in enumerate(numbers):
        del_board = []
        for n_board, board in enumerate(board_sets):

            for row in board:
                row.discard(n)
                if len(row) == 0:
                    del_board.append(n_board)
                    last_board = board
        
        if len(del_board)>0:
            board_tmp = board_sets.copy()
            
            for board in set(del_board):
                board_sets.remove(board_tmp[board])
            
            if len(board_sets) == 0:
                keep_going = False

        if keep_going:
            continue
        else:
            break
            
            
    remainders = set(i for row in last_board for i in row)
    answer = sum(remainders)*n
    return answer


print('Answer:', dec4_part2(input_4))
timer_function(func=dec4_part2,data=input_4, n_run=10)

Answer: 12635
Runtime: 0.007802414894104004 s


# December 5

In [19]:
input_test = [
    '0,9 -> 5,9',
    '8,0 -> 0,8',
    '9,4 -> 3,4',
    '2,2 -> 2,1',
    '7,0 -> 7,4',
    '6,4 -> 2,0',
    '0,9 -> 2,9',
    '3,4 -> 1,4',
    '0,0 -> 8,8',
    '5,5 -> 8,2']
input_5 = scrape_data(day=5)

In [20]:
# Part 1
def data_prep(data):
    data_clean = [tuple(map(int, row.replace(' -> ', ',').split(','))) for row in data]
    
    n_size = np.max(data_clean) + 1
    
    grid = np.zeros((n_size,n_size))
    
    return data_clean, grid

def add_line(coords):
    if coords[0] == coords[2]:
        x = coords[0]
        y1 = min(coords[1], coords[3])
        y2 = max(coords[1], coords[3])

        for yi in range(y1,y2+1):
            grid[yi][x] += 1
    else:
        y = coords[1]
        x1 = min(coords[0], coords[2])
        x2 = max(coords[0], coords[2])

        for xi in range(x1,x2+1):
            grid[y][xi] += 1

def dec5_part1(data):
    for coord in line_coords:
        add_line(coords=coord)
    return np.sum(grid > 1)


line_coords, grid = data_prep(input_5)
# ONly horizontal/vertical
line_coords = [row for row in line_coords if (row[0]==row[2]) | (row[1]==row[3])]

print('Answer:', dec5_part1(line_coords))
timer_function(func=dec5_part1,data=input_5, n_run=10)

Answer: 8060
Runtime: 0.05161151885986328 s


In [21]:
# Part 2
def add_line_diag(coords):
    x1, y1, x2, y2 = coords
    x_step = np.sign(x2 - x1)
    y_step = np.sign(y2 - y1)
    

    if x1 == x2:
        for yi in range(y1, y2 + y_step, y_step):
            grid[yi][x1] += 1
    elif y1 == y2:
        for xi in range(x1, x2 + x_step, x_step):
            grid[y1][xi] += 1
    else:
        for xi,yi in zip(range(x1, x2 + x_step, x_step), range(y1, y2 + y_step, y_step)):
            grid[yi][xi] += 1
            

def dec5_part2(data):
    for coord in line_coords:
        add_line_diag(coords=coord)
    return np.sum(grid > 1)

line_coords, grid = data_prep(input_5)
print('Answer:', dec5_part2(line_coords))
timer_function(func=dec5_part2,data=input_5, n_run=10)

Answer: 21577
Runtime: 0.09042098522186279 s


# December 6

In [22]:
input_test = list(map(int, ['3,4,3,1,2'][0].split(',')))
input_6 = list(map(int, scrape_data(day=6)[0].split(',')))
input_test

[3, 4, 3, 1, 2]

In [23]:
# Part 1
def dec6_part1(fish_pop_initial):
    days = 80
    fish_pop = np.array(fish_pop_initial.copy(), dtype=int)
    for day in range(1, days + 1):
        mask_reset = fish_pop == 0

        fish_pop -= 1*(fish_pop >= 0)

        fish_pop[mask_reset] = 6
        fish_pop = np.append(fish_pop, np.sum(mask_reset, dtype=int)*[8])

    return fish_pop.astype(int)

print('Answer:', len(dec6_part1(input_6)))
timer_function(func=dec6_part1,data=input_6,n_run=10)

Answer: 362346
Runtime: 0.04851164817810059 s


In [24]:
# Part 2
def dec6_part2(fish_pop_initial):
    days = 256
    # Initial states of fish
    fish_states = np.zeros([9])
    for fish in fish_pop_initial:
        fish_states[fish] += 1
    
    # fish evolving logic
    evolve_map = [
        np.array([-1, 0, 0, 0, 0, 0, 1, 0, 1]),
        np.array([1, -1, 0, 0, 0, 0, 0, 0, 0]),
        np.array([0, 1, -1, 0, 0, 0, 0, 0, 0]),
        np.array([0, 0, 1, -1, 0, 0, 0, 0, 0]),
        np.array([0, 0, 0, 1, -1, 0, 0, 0, 0]),
        np.array([0, 0, 0, 0, 1, -1, 0, 0, 0]),
        np.array([0, 0, 0, 0, 0, 1, -1, 0, 0]),
        np.array([0, 0, 0, 0, 0, 0, 1, -1, 0]),
        np.array([0, 0, 0, 0, 0, 0, 0, 1, -1])]
    
    # fish population evolution
    for day in range(1, days + 1):
        for state, count in enumerate(fish_states):
            fish_states = fish_states + count*evolve_map[state]
        
    return np.sum(fish_states)

print('Answer:', dec6_part2(input_6))
timer_function(func=dec6_part2,data=input_6,n_run=10)

Answer: 1639643057051.0
Runtime: 0.005440831184387207 s


# December 7

In [25]:
input_test = list(map(int,['16,1,2,0,4,2,7,1,2,14'][0].split(',')))
input_7 = list(map(int,scrape_data(7)[0].split(',')))
print('Input length:', len(input_7))
print('Test input:', input_test)

Input length: 1000
Test input: [16, 1, 2, 0, 4, 2, 7, 1, 2, 14]


In [26]:
def dec7_part1(data):
    p = np.array(data)
    pos_min = int(np.median(p))
    fuel_min = np.sum(np.abs(p - pos_min))
    
    return pos_min, fuel_min

min_pos, min_fuel = dec7_part1(input_7)
print(f'Optimal position {min_pos} cost a fuel total of {min_fuel}')
timer_function(func=dec7_part1,data=input_7,n_run=10)

Optimal position 331 cost a fuel total of 349769
Runtime: 9.89675521850586e-05 s


In [27]:
def consecutive_sum(n):
    answer = n*(n + 1)/2
    return answer

def dec7_part2(data):
    p = np.array(data)
    pos = range(np.min(data), np.max(data) + 1)
    dp = [consecutive_sum(np.abs(p - i)) for i in pos]
    fuel = np.sum(dp, axis=1)
    
    return pos[np.argmin(fuel)], np.min(fuel)

min_pos, min_fuel = dec7_part2(input_7)
print(f'Optimal position at {min_pos} cost a fuel total of {min_fuel}')
timer_function(func=dec7_part2,data=input_7,n_run=10)

Optimal position at 479 cost a fuel total of 99540554.0
Runtime: 0.026906037330627443 s


In [28]:
input_test = [
    'be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |fdgacbe cefdb cefbgd gcbe',
    'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |fcgedb cgb dgebacf gc',
    'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |cg cg fdcagb cbg',
    'fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |efabcd cedba gadfec cb',
    'aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |gecf egdcabf bgf bfgea',
    'fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |gebdcfa ecba ca fadegcb',
    'dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |cefg dcbef fcge gbcadfe',
    'bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |ed bcgafe cdgba cbgef',
    'egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |gbdfcae bgc cg cgb',
    'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |fgae cfgab fg bagce']
input_test = [i.split(' |') for i in input_test]
input_8=scrape_data(8)
input_8 = [i.split(' | ') for i in input_8]
print('Input length:', len(input_7))
print('Test input:', input_test)

Input length: 1000
Test input: [['be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb', 'fdgacbe cefdb cefbgd gcbe'], ['edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec', 'fcgedb cgb dgebacf gc'], ['fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef', 'cg cg fdcagb cbg'], ['fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega', 'efabcd cedba gadfec cb'], ['aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga', 'gecf egdcabf bgf bfgea'], ['fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf', 'gebdcfa ecba ca fadegcb'], ['dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf', 'cefg dcbef fcge gbcadfe'], ['bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd', 'ed bcgafe cdgba cbgef'], ['egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg', 'gbdfcae bgc cg cgb'], ['gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc', 'fgae cfgab fg bagce']]


In [29]:
digit_dct = {
    0: ('a','b','c','e','f','g'),
    1: ('c','f'),
    2: ('a','c','d','e','g'),
    3: ('a','c','d','f','g'),
    4: ('b','c','d','f'),
    5: ('a','b','d','f','g'),
    6: ('a','b','d','e','f','g'),
    7: ('a','c','f'),
    8: ('a','b','c','d','e','f','g'),
    9: ('a','b','c','d','f','g')}
digit_cnt = dict([(i,len(digit_dct[i])) for i in range(9+1)])

unique_digit_cnt = [1,4,7,8]
unique_len = set([digit_cnt[d] for d in unique_digit_cnt])

def dec8_part1(data):
    value_cnt = 0
    for segment in data:
        value_cnt += np.sum([(len(o) in unique_len) for o in segment[1].split(' ')])
    
    return value_cnt

print('Solution part 1:', dec8_part1(input_8))
timer_function(func=dec8_part1,data=input_8,n_run=10)

Solution part 1: 318
Runtime: 0.0012053728103637695 s


In [30]:
# 1: 2:4, 3:3, 5:4, 0:4, 6:5, 9:4
# 7: 2:3, 3:2, 5:3, 0:3, 6:4, 9:3
# 4: 2:3, 3:2, 5:2, 0:3, 6:3, 9:2
        
def dec8_part2(data):
    digit_sum = 0
    for line in data:
        segment = [list(d) for d in line[0].split(' ')]
        output = [set(list(d)) for d in line[1].split(' ')]
        digits = sorted([set(d) for d in segment], key=len)

        # known codes
        digit_code = {1: digits[0], 7: digits[1], 4: digits[2], 8: digits[-1]}
        
        # codes with length 5: (2,3,5)
        for d in range(3,6):
            d1 = len(digits[d] - digit_code[1])
            d4 = len(digits[d] - digit_code[4])
            if d1 == 3:
                digit_code[3] = digits[d]
            elif d4 == 3:
                digit_code[2] = digits[d]
            else:
                digit_code[5] = digits[d]
                
        # codes with length 6: (0,6,9)
        for d in range(6,9):
            d1 = len(digits[d] - digit_code[1])
            d4 = len(digits[d] - digit_code[4])
            if d1 == 5:
                digit_code[6] = digits[d]
            elif d4 == 2:
                digit_code[9] = digits[d]
            else:
                digit_code[0] = digits[d]
                
        digit_decifer = [digit_code[i] for i in range(9+1)] # letter-codes ordered after numerical value
        digit = int(''.join([str(digit_decifer.index(o)) for o in output]))
        digit_sum += digit

    return digit_sum

print('Solution part 2:', dec8_part2(input_8))
timer_function(func=dec8_part2,data=input_8,n_run=10)

Solution part 2: 996280
Runtime: 0.0027011871337890626 s


In [31]:
input_test = [
    '2199943210',
    '3987894921',
    '9856789892',
    '8767896789',
    '9899965678']
input_test = [list(map(int,list(row))) for row in input_test]
input_9 = [list(map(int,list(row))) for row in scrape_data(day=9)]
print('Input length:', len(input_9))
print('Test input:\n', np.array(input_test))

Input length: 100
Test input:
 [[2 1 9 9 9 4 3 2 1 0]
 [3 9 8 7 8 9 4 9 2 1]
 [9 8 5 6 7 8 9 8 9 2]
 [8 7 6 7 8 9 6 7 8 9]
 [9 8 9 9 9 6 5 6 7 8]]


In [32]:
def dec9_part1(data):
    row_diff_left= np.diff(data, axis=1, prepend=999)
    row_diff_right = -1*np.diff(data, axis=1, append=999)
    col_diff_up= np.diff(data, axis=0, prepend=999)
    col_diff_down = -1*np.diff(data, axis=0, append=999)
    
    min_loc = (row_diff_right < 0)*(row_diff_left < 0)*(col_diff_up < 0)*(col_diff_down < 0)
    min_coords = np.argwhere(np.isin(min_loc, 1))
    answer = np.sum(min_loc*data + min_loc*1)

    return answer, min_loc,min_coords

solution, min_loc, min_coords = dec9_part1(input_9)
print('Solution part 1:', solution)
timer_function(func=dec9_part1, data=input_9, n_run=10)

Solution part 1: 537
Runtime: 0.0036036491394042967 s


In [33]:
def add_padding(array, num=9):
    padded_arr = np.ones([len(array) + 2, len(array[0]) + 2])*num
    for i, row in enumerate(array):
        padded_arr[i+1][1:-1] = padded_arr[i+1][1:-1]*0 + row
        
    return padded_arr

grid_padded = add_padding(input_9, 9) 
grid_padded = (grid_padded < 9)*1

In [34]:
def check_neighboors(coord_ini, grid, remaining_coords, already_checked_coords):
    x0 = coord_ini[0]
    y0 = coord_ini[1]
    neighbors = [(x0, y0+1), (x0, y0-1), (x0+1, y0), (x0-1, y0)]

    already_checked_coords.add((x0,y0))
    for coord in neighbors:
        x = coord[0]
        y = coord[1]
        grid_val = grid[y][x]
        
        if grid_val == 1:
            remaining_coords.append((x,y))
        
    remaining_coords = [coord for coord in remaining_coords if coord not in already_checked_coords]
    
    return remaining_coords, already_checked_coords


basin_centers = [tuple([min_coords[i][1]+1, min_coords[i][0]+1]) for i in range(len(min_coords))]
already_checked_coords = set()
basin_sizes = []
for basin in basin_centers:
    already_checked_coords = set()
    remaining_coords = [basin]
    while len(remaining_coords) > 0:
        remaining_coords , already_checked_coords = check_neighboors(coord_ini=remaining_coords[0],
                                                                     grid=grid_padded,
                                                                     remaining_coords = remaining_coords,
                                                                     already_checked_coords=already_checked_coords)
    basin_sizes.append(len(already_checked_coords))

print('Answer:', np.product(sorted(basin_sizes)[-3:]))

Answer: 1142757


# December 10

In [35]:
input_test = [
    '[({(<(())[]>[[{[]{<()<>>',
    '[(()[<>])]({[<{<<[]>>(',
    '{([(<{}[<>[]}>{[]{[(<()>',
    '(((({<>}<{<{<>}{[]{[]{}',
    '[[<[([]))<([[{}[[()]]]',
    '[{[{({}]{}}([{[{{{}}([]',
    '{<[[]]>}<{[{[{[]{()[[[]',
    '[<(<(<(<{}))><([]([]()',
    '<{([([[(<>()){}]>(<<{{',
    '<{([{{}}[<[[[<>{}]]]>[]]']

input_10 = scrape_data(day=10)
print('Input length:', len(input_10))
print('Test input:\n', np.array(input_test))

Input length: 94
Test input:
 ['[({(<(())[]>[[{[]{<()<>>' '[(()[<>])]({[<{<<[]>>('
 '{([(<{}[<>[]}>{[]{[(<()>' '(((({<>}<{<{<>}{[]{[]{}'
 '[[<[([]))<([[{}[[()]]]' '[{[{({}]{}}([{[{{{}}([]'
 '{<[[]]>}<{[{[{[]{()[[[]' '[<(<(<(<{}))><([]([]()'
 '<{([([[(<>()){}]>(<<{{' '<{([{{}}[<[[[<>{}]]]>[]]']


In [36]:
# Part 1
def remove_closed_chunks(row):
    len_prev = len(row)
    len_new = len_prev - 1
    while len_new < len_prev:
        len_prev = len_new
        row = row.replace('()', '').replace('[]', '').replace('{}', '').replace('<>', '')
        len_new = len(row)
        
    return row
    
def dec10_part1(data):
    rows_corrupt = []
    rows_incomplete = []
    corrupt_score = 0
    score_map = {')': 3, ']': 57, '}': 1197, '>': 25137}
    
    for row in data:
        # filter out closed chunks
        row_incomplete = remove_closed_chunks(row)
        # remove all open chars and find first closing char
        row_score = row_incomplete.replace('(', '').replace('[', '').replace('{', '').replace('<', '')

        if len(row_score) == 0:
            rows_incomplete.append(row_incomplete)
        else:
            rows_corrupt.append(row)
            corrupt_score += score_map[row_score[0]]
        

    return rows_incomplete, rows_corrupt, corrupt_score
        
#input_test_trim = remove_incomplete(input_test)
keep_rows, corrupt_rows, corrupt_score = dec10_part1(input_10)
print('Solution:', corrupt_score)

Solution: 345441


In [37]:
# Part 2
def calculate_score(missing_chars):
    score_map = {'(': 1, '[': 2, '{': 3, '<': 4}
    score = 0
    
    for char in missing_chars:
        score = score*5 + score_map[char]
        
    return score

def dec10_part2(missing_lines):
    missing_lines_ordered = [row[::-1] for row in missing_lines]

    complete_score = []
   
    for line in missing_lines_ordered:
        complete_score.append(calculate_score(line))

    score_middle = np.median(complete_score)
    
    return score_middle


keep_rows, corrupt_rows, corrupt_score = dec10_part1(input_10)
dec10_part2(keep_rows)

3235371166.0

# December 11

In [38]:
input_test = [
    '11111',
'19991',
'19191',
'19991',
'11111']
input_test = [
    '5483143223',
'2745854711',
'5264556173',
'6141336146',
'6357385478',
'4167524645',
'2176841721',
'6882881134',
'4846848554',
'5283751526'
]
input_test = [list(map(int, row)) for row in input_test]
input_11 = scrape_data(day=11)
input_11 = [list(map(int, row)) for row in input_11]
print('Input length:', len(input_11))
print('Test input:\n', np.array(input_test))

Input length: 10
Test input:
 [[5 4 8 3 1 4 3 2 2 3]
 [2 7 4 5 8 5 4 7 1 1]
 [5 2 6 4 5 5 6 1 7 3]
 [6 1 4 1 3 3 6 1 4 6]
 [6 3 5 7 3 8 5 4 7 8]
 [4 1 6 7 5 2 4 6 4 5]
 [2 1 7 6 8 4 1 7 2 1]
 [6 8 8 2 8 8 1 1 3 4]
 [4 8 4 6 8 4 8 5 5 4]
 [5 2 8 3 7 5 1 5 2 6]]


In [39]:
# Part 1
def boost_neigbors(x, y, grid):
    grid[y-1:y+2, x - 1] += 1
    grid[y-1:y+2, x - 0] += 1
    grid[y-1:y+2, x + 1] += 1
    
    return grid


def run_step(grid):
    n_row = len(grid) - 1
    n_col = len(grid[0]) - 1
    grid_old = grid.copy()
    grid_new = grid.copy() + 1
    
    while np.array_equal(grid_new, grid_old, equal_nan=True) is False:

        grid_old = grid_new.copy()
        
        for yi in range(1, n_row):
            for xi in range(1, n_col):
                if grid_new[yi, xi] > 9:
                    grid_new = boost_neigbors(x=xi, y=yi, grid=grid_new)
                    grid_new[yi, xi] = np.nan

    grid_new[np.isnan(grid_new)] = 0


    return grid_new

def run_n_steps(grid):
    n_steps=100
    flash_sum = 0
    for n in range(n_steps):
        grid = run_step(grid)
        mask_flash = grid == 0
        flash_sum += np.sum(mask_flash)

    return flash_sum, grid


input_padded = add_padding(input_11)
answer, grid = run_n_steps(input_padded)
print('PArt 1 answer:', answer)
timer_function(func=run_n_steps, data=input_padded, n_run=10)

PArt 1 answer: 1686
Runtime: 0.02660651206970215 s


In [40]:
# Part 2
def flash_simul(grid):
    flashes = 0
    n_all = (len(grid) - 2)**2
    step = 0
    
    while (flashes != n_all):
        grid = run_step(grid)
        flashes = np.sum(grid == 0)

        step += 1
        
    return step

grid = add_padding(input_11,-999)

steps = flash_simul(grid)
print('Part 2 solution:', steps)
timer_function(func=flash_simul, data=grid, n_run=10)

Part 2 solution: 360
Runtime: 0.08673608303070068 s


# December 12

In [41]:
input_test = ['start-A',
'start-b',
'A-c',
'A-b',
'b-d',
'A-end',
'b-end']

# input_test = ['dc-end',
# 'HN-start',
# 'start-kj',
# 'dc-start',
# 'dc-HN',
# 'LN-dc',
# 'HN-end',
# 'kj-sa',
# 'kj-HN',
# 'kj-dc']

# input_test = ['fs-end',
# 'he-DX',
# 'fs-he',
# 'start-DX',
# 'pj-DX',
# 'end-zg',
# 'zg-sl',
# 'zg-pj',
# 'pj-he',
# 'RW-he',
# 'fs-DX',
# 'pj-RW',
# 'zg-RW',
# 'start-pj',
# 'he-WI',
# 'zg-he',
# 'pj-fs',
# 'start-RW']

input_test = [line.split('-') for line in input_test]
input_12 = scrape_data(day=12)
input_12 = [line.split('-') for line in input_12]
print('Input length:', len(input_12))
print('Test input:\n', np.array(input_test))

Input length: 22
Test input:
 [['start' 'A']
 ['start' 'b']
 ['A' 'c']
 ['A' 'b']
 ['b' 'd']
 ['A' 'end']
 ['b' 'end']]


In [42]:
import random

def create_neighbor(data):
    graph = {}
    for path in data:
        n = graph.get(path[0], list())
        n.append(path[1])
        graph[path[0]] = n
        
        n = graph.get(path[1], list())
        n.append(path[0])
        graph[path[1]] = n
    
    # remove end node
    graph.pop('end')
    
    for node in graph.copy():
        # remove start as a neighbor
        if node != 'start' and 'start' in graph[node]:
            n = graph[node]
            graph[node].remove('start')
            graph[node] = n
            
        graph[node] = sorted(graph[node])
 
    return graph


graph = create_neighbor(input_12)
graph

{'GC': ['ca', 'end', 'ky', 'lk', 'zi', 'zv'],
 'zi': ['GC', 'ca', 'lk'],
 'zv': ['FU', 'GC', 'QQ', 'ca', 'end'],
 'lk': ['FU', 'GC', 'ca', 'iv', 'zi'],
 'ca': ['FU', 'GC', 'iv', 'lk', 'zi', 'zv'],
 'ky': ['GC'],
 'FU': ['ca', 'end', 'iv', 'lk', 'zv'],
 'iv': ['FU', 'ca', 'lk'],
 'start': ['iv', 'lk', 'zi'],
 'QQ': ['zv']}

In [43]:
# Part 1 - recursion
def add_node_part1(adj_lst, node_current, node_explored, path_current):
    global path_count

    for node in adj_lst[node_current]:
        new_explored = node_explored.copy()
        new_path = path_current.copy()
        new_path.append(node)
        
        if node == 'end':
            path_count += 1 
        elif node not in node_explored:
            if node.islower():
                new_explored.add(node)
            # recursive call
            add_node_part1(adj_lst, node_current=node, node_explored=new_explored, path_current=new_path)
        else:
            pass

    return path_count
        

def find_paths(adj_lst, start_node='start', end_node='end'):
    explored = set() # explored
    path = [start_node]
    explored.add(start_node)
    count = add_node_part1(adj_lst, node_current=start_node, node_explored=explored, path_current=path)
    
    return count

global path_count
path_count = 0
find_paths(graph, start_node='start', end_node='end')


5252

In [44]:
# Breadth first Search algo
def bfs(adj_lst, start_node='start', end_node='end'):
    queue = [] # queue
    explored = set() # explored
    path_count = 0

    queue.append(start_node)
    
    while len(queue) > 0:
        node = queue.pop()
                    
        for neighbor in adj_lst[node]:
            if neighbor == end_node:
                path_count += 1
            elif neighbor.islower() and (neighbor not in explored):
                explored.add(neighbor)
                queue.append(neighbor)
            elif neighbor.isupper():
                queue.append(neighbor)
                
    return path_count

bfs(graph, start_node='start', end_node='end')

10

In [45]:
# Part 2
def add_node_part2(adj_lst, node_current, path_current, cave_cnt):
    global path_count

    for node in adj_lst[node_current]:
        new_path = path_current.copy()
        new_path.append(node)
        new_cave_cnt = cave_cnt.copy()
        
        if node == 'end':
            path_count += 1
        elif node.islower():
            new_cave_cnt[node] += 1

            if new_cave_cnt[node] == 2:
                explored = set()
                for cave in new_cave_cnt:
                    if new_cave_cnt[cave] > 0:
                        explored.add(cave)

                # recursive call to part 1
                add_node_part1(adj_lst, node_current=node, node_explored=explored, path_current=new_path)
            else:
                add_node_part2(adj_lst, node_current=node, path_current=new_path, cave_cnt=new_cave_cnt)
        else:
            # recursive call
            add_node_part2(adj_lst, node_current=node, path_current=new_path, cave_cnt=new_cave_cnt)


    return path_count
        

def find_paths_part2(adj_lst, start_node='start', end_node='end'):
    path = [start_node]
    
    cave_cnt = {}
    for node in adj_lst:
        if node.islower() and node not in [start_node, end_node]:
            cave_cnt[node] = 0
  
    count = add_node_part2(adj_lst, node_current=start_node, path_current=path, cave_cnt=cave_cnt)
    
    return count

global path_count
path_count = 0

find_paths_part2(graph, start_node='start', end_node='end')


147784

# December 13

In [46]:
input_test = ['6,10',
'0,14',
'9,10',
'0,3',
'10,4',
'4,11',
'6,0',
'6,12',
'4,1',
'0,13',
'10,12',
'3,4',
'3,0',
'8,4',
'1,10',
'2,14',
'8,10',
'9,0',
'',
'fold along y=7',
'fold along x=5']

input_13 = scrape_data(day=13)
print('Input length:', len(input_13))
print('Test input:\n', np.array(input_test))

Input length: 1033
Test input:
 ['6,10' '0,14' '9,10' '0,3' '10,4' '4,11' '6,0' '6,12' '4,1' '0,13'
 '10,12' '3,4' '3,0' '8,4' '1,10' '2,14' '8,10' '9,0' '' 'fold along y=7'
 'fold along x=5']


In [47]:
def data_parse_13(data):
    
    dots = [tuple(map(int, x.split(','))) for x in data if len(x) > 0 and x[0] != 'f']
    folds = [list(map(str, x.split('='))) for x in data if len(x) > 0 and x[0] == 'f']
    folds = [(f[0][-1], int(f[1])) for f in folds]
    
    return dots, folds

dots, folds = data_parse_13(input_test)
print(dots)
print(folds)

[(6, 10), (0, 14), (9, 10), (0, 3), (10, 4), (4, 11), (6, 0), (6, 12), (4, 1), (0, 13), (10, 12), (3, 4), (3, 0), (8, 4), (1, 10), (2, 14), (8, 10), (9, 0)]
[('y', 7), ('x', 5)]


In [48]:
# PArt 1
def dec13_part1(dots, folds):
    dot_grid = set(dots)
    dots_current = dots.copy()
    
    for n_fold, fold in enumerate(folds, 1):
        axis = fold[0]
        coord = fold[1]
        
        if axis == 'x':
            [dot_grid.add((2*coord - d[0], d[1])) for d in dots_current if d[0] > coord]
            [dot_grid.remove(d) for d in dots_current if d[0] > coord]
        else:
            [dot_grid.add((d[0], 2*coord - d[1])) for d in dots_current if d[1] > coord]
            [dot_grid.remove(d) for d in dots_current if d[1] > coord]
        
        dots_current = dot_grid.copy()
        print(f'Fold {n_fold}, {fold}:', len(dot_grid))
    
    return dot_grid
        
dots, folds = data_parse_13(input_13)
final_dot_grid = dec13_part1(dots, folds)

Fold 1, ('x', 655): 850
Fold 2, ('y', 447): 698
Fold 3, ('x', 327): 588
Fold 4, ('y', 223): 494
Fold 5, ('x', 163): 398
Fold 6, ('y', 111): 344
Fold 7, ('x', 81): 280
Fold 8, ('y', 55): 231
Fold 9, ('x', 40): 190
Fold 10, ('y', 27): 155
Fold 11, ('y', 13): 122
Fold 12, ('y', 6): 102


In [49]:
# Part 2
n_col = np.max([dot[0] for dot in final_dot_grid])
n_row = np.max([dot[1] for dot in final_dot_grid])

dot_grid = np.zeros([n_row+1, n_col+1])
for dot in final_dot_grid:
    dot_grid[dot[1]][dot[0]] = 1

idx = np.where(np.sum(dot_grid,axis=0) == 0)
idx = np.insert(idx, [0], 0)
idx = np.append(idx, [n_col+1])

for i in range(len(idx)-1):
    print(dot_grid[:, idx[i]:idx[i+1]])

# A H G C P G A U

[[0. 1. 1. 0.]
 [1. 0. 0. 1.]
 [1. 0. 0. 1.]
 [1. 1. 1. 1.]
 [1. 0. 0. 1.]
 [1. 0. 0. 1.]]
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]
[[0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 1.]]
[[0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 0.]]
[[0. 1. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 1. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
[[0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 1.]]
[[0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 0.]]


# December 14

In [50]:
input_test = ['NNCB',
              '',
                'CH -> B',
                'HH -> N',
                'CB -> H',
                'NH -> C',
                'HB -> C',
                'HC -> B',
                'HN -> C',
                'NN -> C',
                'BH -> H',
                'NC -> B',
                'NB -> B',
                'BN -> B',
                'BB -> N',
                'BC -> B',
                'CC -> N',
                'CN -> C']

input_14 = scrape_data(day=14)
print('Input length:', len(input_14))
print('Test input:\n', np.array(input_test))

Input length: 102
Test input:
 ['NNCB' '' 'CH -> B' 'HH -> N' 'CB -> H' 'NH -> C' 'HB -> C' 'HC -> B'
 'HN -> C' 'NN -> C' 'BH -> H' 'NC -> B' 'NB -> B' 'BN -> B' 'BB -> N'
 'BC -> B' 'CC -> N' 'CN -> C']


In [51]:
def input_parse_14_part1(data):
    seq_start = data[0]
    rules = [i.split(' -> ') for i in data[2:]]
    rules_triple = {}
    for rule in rules.copy():
        char_1 = rule[0][0]
        char_2 = rule[0][1]
        char_new = str.lower(rule[1])
        
        # A single char might appear 3 consecutive times at most
        if char_1 == char_2:
            rules_triple[3*char_1] = char_1 + char_new + char_2 + char_new +char_2

        rules_triple[rule[0]] = char_1 + char_new + char_2

    return seq_start, rules_triple
input_parse_14_part1(input_test)

('NNCB',
 {'CH': 'CbH',
  'HHH': 'HnHnH',
  'HH': 'HnH',
  'CB': 'ChB',
  'NH': 'NcH',
  'HB': 'HcB',
  'HC': 'HbC',
  'HN': 'HcN',
  'NNN': 'NcNcN',
  'NN': 'NcN',
  'BH': 'BhH',
  'NC': 'NbC',
  'NB': 'NbB',
  'BN': 'BbN',
  'BBB': 'BnBnB',
  'BB': 'BnB',
  'BC': 'BbC',
  'CCC': 'CnCnC',
  'CC': 'CnC',
  'CN': 'CcN'})

In [52]:
def dec14_part1(data, steps):
    seq, rules = input_parse_14_part1(data=data)
    
    # Insert between pairs
    for step in range(1, steps + 1):
        for rule in rules:
            seq = seq.replace(rule, rules[rule])
            
        seq = str.upper(seq)
        print(f'Step {step}: polymer length {len(seq)}')
    
    # find min/max
    seq_lst = list(seq)
    char_cnt_dct = {}
    for char in np.unique(seq_lst):
        char_cnt_dct[char] = seq_lst.count(char)
    
    print(f'Count of characters in polymer \n', char_cnt_dct)
    cnt_min = char_cnt_dct[min(char_cnt_dct, key=char_cnt_dct.get)]
    cnt_max = char_cnt_dct[max(char_cnt_dct, key=char_cnt_dct.get)]
    
    answer = cnt_max - cnt_min
    
    return answer

answer = dec14_part1(data=input_test, steps=10)
print(f'Difference betwen max and min character counts:', answer)

Step 1: polymer length 7
Step 2: polymer length 13
Step 3: polymer length 25
Step 4: polymer length 49
Step 5: polymer length 97
Step 6: polymer length 193
Step 7: polymer length 385
Step 8: polymer length 769
Step 9: polymer length 1537
Step 10: polymer length 3073
Count of characters in polymer 
 {'B': 1749, 'C': 298, 'H': 161, 'N': 865}
Difference betwen max and min character counts: 1588


In [53]:
 # Part 2 - Dont iterate htorugh series, but just count evolutions
def input_parse_14_part2(data):
    seq_start = data[0]
    rules = [i.split(' -> ') for i in data[2:]]
    rules = dict(sorted([(r[0], r[1]) for r in rules]))

    return seq_start, rules

def pair_trasform(rules):
    transform = {}
    for r in rules:
        transform[r] = [r[0] + rules[r], rules[r] + r[1]]
    return transform

def initialize_char_count(seq, rules):
    chars = ''
    for r in rules:
        chars += r
    char_cnt = {}
    for char in np.unique(list(chars)):
        char_cnt[char] = 0
    
    for char in seq:
        char_cnt[char] +=1
        
    return char_cnt

def initialize_pair_count(seq, rules):
    pair_cnt = {}
    for rule in rules:
        pair_cnt[rule] = 0
    
    pairs_initial = [seq[i:i+2] for i in range(len(seq)-1)]
    for pair in pairs_initial:
        pair_cnt[pair] += 1
        
    return pair_cnt

def dec14_part2(data, steps):
    seq, rules = input_parse_14_part2(data)
    pair_trans = pair_trasform(rules)
    char_cnt = initialize_char_count(seq, rules)
    pair_cnt = initialize_pair_count(seq, rules)

    for step in range(1, steps + 1):
        pair_count_current = pair_cnt.copy()
        for pair in pair_cnt:
            n_pair_current = pair_count_current[pair]
            char_cnt[rules[pair]] += 1*n_pair_current
            pair_cnt[pair] -= 1*n_pair_current
            for new_pair in pair_trans[pair]:
                pair_cnt[new_pair] += 1*n_pair_current

    
    cnt_min = char_cnt[min(char_cnt, key=char_cnt.get)]
    cnt_max = char_cnt[max(char_cnt, key=char_cnt.get)]
    polymer_lenght = sum([char_cnt[key] for key in char_cnt])
    
    answer = cnt_max - cnt_min
    return answer, polymer_lenght, char_cnt

answer, polymer_lenght, char_cnt = dec14_part2(data=input_14, steps=40)
print(f'Polymer has length {polymer_lenght:,} with a distribution of characters:\n', char_cnt)
print(f'Max char - min char count: {answer:,}')

Polymer has length 20,890,720,927,745 with a distribution of characters:
 {'B': 2110238569763, 'C': 721076812001, 'F': 3828470546714, 'H': 1394050790085, 'K': 2058813349699, 'N': 2572922913435, 'O': 2127526166530, 'P': 2578458607681, 'S': 2903118851587, 'V': 596044320250}
Max char - min char count: 3,232,426,226,464


# December 15

In [54]:
input_test = [
    '1163751742',
    '1381373672',
    '2136511328',
    '3694931569',
    '7463417111',
    '1319128137',
    '1359912421',
    '3125421639',
    '1293138521',
    '2311944581'
]
input_test = np.array([list(map(int, row)) for row in input_test])
input_15 = scrape_data(day=15)
input_15 = np.array([list(map(int, row)) for row in input_15])
print('Input length:', len(input_15))
print('Test input:\n', np.array(input_test))

Input length: 100
Test input:
 [[1 1 6 3 7 5 1 7 4 2]
 [1 3 8 1 3 7 3 6 7 2]
 [2 1 3 6 5 1 1 3 2 8]
 [3 6 9 4 9 3 1 5 6 9]
 [7 4 6 3 4 1 7 1 1 1]
 [1 3 1 9 1 2 8 1 3 7]
 [1 3 5 9 9 1 2 4 2 1]
 [3 1 2 5 4 2 1 6 3 9]
 [1 2 9 3 1 3 8 5 2 1]
 [2 3 1 1 9 4 4 5 8 1]]


In [55]:
# PArt 1 - Dijkstra's algorithm
def get_neigbor_nodes(index, grid):
    # No diagonal neighbors
    n_row = len(grid) - 1
    n_col = len(grid[0]) - 1
    x = index[1]
    y = index[0]
    
    neighbors = [(y - 1, x), (y + 1, x), (y, x - 1), (y, x + 1)]
    neighbors_valid = [i for i in neighbors if (i[0] >= 0) and (i[0] <= n_row) and (i[1] >= 0) and (i[1] <= n_col)]
    
    return neighbors_valid

grid = input_test.copy()
print(grid)
get_neigbor_nodes((9,9), grid)

[[1 1 6 3 7 5 1 7 4 2]
 [1 3 8 1 3 7 3 6 7 2]
 [2 1 3 6 5 1 1 3 2 8]
 [3 6 9 4 9 3 1 5 6 9]
 [7 4 6 3 4 1 7 1 1 1]
 [1 3 1 9 1 2 8 1 3 7]
 [1 3 5 9 9 1 2 4 2 1]
 [3 1 2 5 4 2 1 6 3 9]
 [1 2 9 3 1 3 8 5 2 1]
 [2 3 1 1 9 4 4 5 8 1]]


[(8, 9), (9, 8)]

In [56]:
def dijkstra(graph, source, target=None):
    Q = set()
    dist = {}
    prev = {}
    value = {}
    
    for idx_row, row in enumerate(graph):
        for idx_col, col in enumerate(row):
            v = (idx_row, idx_col)
            dist[v] = np.inf
            prev[v] = np.nan
            value[v] = graph[v]
            Q.add(v)

    dist[source] = 0
    
    while len(Q) > 0:
        dist_search = [d for d in dist if d in Q]
        u = min(dist_search, key=dist.get)
        Q.remove(u)
        
        if u == target:
            print('Found target')
            break
        
        neighbors = get_neigbor_nodes(index=u, grid=graph)
        neighbors_in_Q = [n for n in neighbors if n in Q]
        for neighbor in neighbors_in_Q:
            alt = dist[u] + value[neighbor]
            if alt < dist[neighbor]:
                dist[neighbor] = alt
                prev[neighbor] = u
            
    return dist, prev, Q

grid = input_test
t0 = time.time()
dist, prev, Q = dijkstra(graph=grid, source=(0,0), target=(len(grid)-1, len(grid[0])-1))
print('Solution, total risk:', dist[len(grid)-1, len(grid[0])-1])
print('Runtime:', time.time()-t0)

Found target
Solution, total risk: 40
Runtime: 0.007001638412475586


In [57]:
import heapq

def dijkstra_heapq(graph, source, target=None):
    dist = {}
    heap = []
    prev = {}
    value = {}
    
    for idx_row, row in enumerate(graph):
        for idx_col, col in enumerate(row):
            v = (idx_row, idx_col)
            dist[v] = np.inf
            prev[v] = np.nan
            value[v] = graph[v]
            heapq.heappush(heap, (np.inf, v))

    dist[source] = 0
    heapq.heappush(heap, (0, source))
    
    while len(heap) > 0:
        u = heapq.heappop(heap)[1]
        
        if u == target:
            print('Found target')
            break
        
        neighbors = get_neigbor_nodes(index=u, grid=graph)
        for neighbor in neighbors:
            alt = dist[u] + value[neighbor]
            if alt < dist[neighbor]:
                dist[neighbor] = alt
                heapq.heappush(heap, (alt, neighbor))
                prev[neighbor] = u
            
    return dist, prev, heap

grid = input_15
t0 = time.time()
dist, prev, heap = dijkstra_heapq(graph=grid, source=(0,0), target=(len(grid)-1, len(grid[0])-1))
print('Solution, total risk:', dist[len(grid)-1, len(grid[0])-1])
print('Runtime:', time.time()-t0)

Found target
Solution, total risk: 388
Runtime: 0.24706196784973145


In [58]:
# PArt 2
def a_star_heapq(graph, source, target=None):
    Q = []
    g_score = {}
    f_score = {}
    h = {}
    prev = {}
    value = {}
    
    idx_row_max = len(graph) - 1
    idx_col_max = len(graph[0]) - 1

    for idx_row, row in enumerate(graph):
        for idx_col, col in enumerate(row):
            v = (idx_row, idx_col)
            g_score[v] = np.inf
            f_score[v] = np.inf
            prev[v] = np.nan
            value[v] = graph[v]
            h[v] = idx_row_max - idx_row + idx_col_max - idx_col # Manhattan distance
            heapq.heappush(Q, (np.inf, v))
            

    g_score[source] = 0
    f_score[source] = h[source]
    heapq.heappush(Q, (f_score[source], source))
    
    while len(Q) > 0:
        u = heapq.heappop(Q)[1]
        if u == target:
            print('Found target')
            break

        neighbors = get_neigbor_nodes(index=u, grid=graph)
        for neighbor in neighbors:
            alt_g_score = g_score[u] + value[neighbor]
            if alt_g_score < g_score[neighbor]:
                g_score[neighbor] = alt_g_score
                f_score[neighbor] = alt_g_score + h[neighbor]
                prev[neighbor] = u
                heapq.heappush(Q, (f_score[neighbor], neighbor))

    return g_score, prev, Q


t0 = time.time()
grid = input_15
g_score, prev, Q = a_star_heapq(graph=grid, source=(0,0), target=(len(grid)-1, len(grid[0])-1))
print('Solution, total risk:', g_score[len(grid)-1, len(grid[0])-1])
print('Runtime:', time.time()-t0)

Found target
Solution, total risk: 388
Runtime: 0.24657058715820312


In [59]:
grid = input_15
n_keep = len(grid)
grid_prev = grid.copy()
n = 5

for i in range(1, n):
    grid = np.append(grid, grid_prev + 1, axis=0)
    grid[grid > 9] = 1
    grid_prev = grid[-n_keep:, :]

grid_prev = grid.copy()
for i in range(1, n):

    grid = np.append(grid, grid_prev + 1, axis=1)
    grid[grid > 9] = 1
    grid_prev = grid[:, -n_keep:]
    
t0 = time.time()
g_score, prev, Q = a_star_heapq(graph=grid, source=(0,0), target=(len(grid)-1, len(grid[0])-1))
print('Solution, total risk:', g_score[len(grid)-1, len(grid[0])-1])
print('Runtime:', time.time()-t0)

Found target
Solution, total risk: 2819
Runtime: 8.3007972240448


# December 20

In [60]:
# data fetch
def data_prep_20(data):
    output_algo = data[0].replace('.', '0').replace('#', '1')
    input_image = [i.replace('#', '1').replace('.', '0') for i in data[2:]]

    return output_algo, input_image


input_test = ['..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#',
              '',
              '#..#.',
              '#....',
               '##..#',
               '..#..',
               '..###']

input_20 = scrape_data(day=20)
print('Input length:', len(input_20))

output_algo, input_image = data_prep_20(input_20)
output_algo[int('111111111',2)]

Input length: 102


'0'

In [61]:
def add_padding(img, pad_char):
    img_padded = img.copy()
    n_pad = 3
    img_padded = [n_pad*pad_char + row + n_pad*pad_char for row in img_padded]
    
    n_row = len(img_padded[0])
    for _ in range(n_pad):
        img_padded = np.insert(img_padded, 0, pad_char*n_row)
        img_padded = np.append(img_padded, pad_char*n_row)

    return img_padded
    
def fetch_neighbor_inputs(img, idx):
    row = idx[0]
    col = idx[1]
    seq = img[row - 1][col-1:col+2] + img[row][col-1:col+2] + img[row + 1][col-1:col+2]

    return int(seq, 2)

def get_output_image(input_img, pixel_algo, pad_char):
    img_padded = add_padding(input_img, pad_char)
    n_row = len(img_padded)
    n_col = len(img_padded[0])
    
    img_output = []
    for row in range(1, n_row - 1):
        output_row = ''
        for col in range(1, n_col - 1):
            sequence = fetch_neighbor_inputs(img=img_padded, idx=(row, col))
            pixel = pixel_algo[sequence]
            output_row += pixel

        img_output.append(output_row)
    
    return img_output

def dec20_part1(data, steps):
    output_algo, input_image = data_prep_20(data)
    output_image = input_image.copy()
    
    padding_next = '0'

    for step in range(0, steps):
        output_image = get_output_image(input_img = output_image, pixel_algo=output_algo, pad_char=padding_next)
        padding_next = output_algo[int(padding_next*9, 2)]
    
    lit_pixels = np.sum([i.count('1') for i in output_image])
    
    return output_image, lit_pixels

output_image, lit_pixels = dec20_part1(data=input_20, steps=50)
lit_pixels

19012

# December 17

In [62]:
# Part 1

In [63]:
def trajectory_step(position, velocity):
    px = position[0]
    py = position[1]
    vx = velocity[0]
    vy = velocity[1]
    
    px_new = px + vx
    py_new = py + vy
    
    vy_new = vy - 1
    
    if vx != 0:
        vx_new = vx - np.sign(vx)
    else:
        vx_new = 0
    
    return (px_new, py_new), (vx_new, vy_new)


def run_trajectory(pos_0, vel_0, target):
    position = [pos_0]
    velocity = [vel_0]
    target_zone, x_max, y_min = create_target_zone(x_start=target[0][0],
                                                   x_end=target[0][1],
                                                   y_start=target[1][0],
                                                   y_end=target[1][1])
    pos_step = pos_0
    vel_step = vel_0
    
    while True:
        pos_step, vel_step = trajectory_step(pos_step, vel_step)
        
        position.append(pos_step)
        velocity.append(vel_step)
        
        if pos_step in target_zone:
            #print('HIT:', pos_step)
            hit = True
            break
        elif (pos_step[0] > x_max) or (pos_step[1] < y_min):
            #print('MISS:', pos_step)
            hit = False
            break
            
    return position, velocity, hit


def create_target_zone(x_start, x_end, y_start, y_end):
    target_zone = set()
    if x_end > x_start:
        x_min = x_start
        x_max = x_end
    else:
        x_min = x_end
        x_max = x_start
        
    if y_end > y_start:
        y_min = y_start
        y_max = y_end
    else:
        y_min = y_end
        y_max = y_start
    
    for xi in range(x_min, x_max + 1):
        for yi in range(y_min, y_max + 1):
            target_zone.add((xi,yi))
    
    return target_zone, x_max, y_min

def velocity_bounds_x(target):
    xt_min = target[0][0]
    xt_max = target[0][1]
    vx0_min = np.ceil((np.sqrt(1 + 8*xt_min) - 1)/2)
    vx0_max = np.floor(xt_max)

    return int(vx0_min), int(vx0_max)

def velocity_bounds_y(n_min, n_max, target):
    yt_min = target[1][0]
    yt_max = target[1][1]
    vy0_max = int(np.floor((yt_max + n_min*(n_min - 1)/2)/n_min))
    vy0_min = int(np.floor((yt_min + n_max*(n_max - 1)/2)/n_max))
        
    return min(vy0_min, vy0_max), max(vy0_min, vy0_max)

def step_bounds(velocity, target):
    xt_min = target[0][0]
    xt_max = target[0][1]
    yt_max = target[1][1]

    n_min = int(np.ceil((2*velocity + 1 - np.sqrt((2*velocity + 1)**2 - 8*xt_min))/2))
    
    det = (2*velocity + 1)**2 - 8*xt_max
    if det >= 0:
        n_max = int(np.floor((2*velocity + 1 - np.sqrt(det))/2))
    else:
        n_max = np.abs(2*yt_max)

    return n_min,  n_max

def dec17_part1(target):
    vx0_min, vx0_max = velocity_bounds_x(target=target)

    ytop_max = 0
    velocity_best = [0, 0]
    velocity_hit_set = set()
    print('Velocity range x:', vx0_min, vx0_max)
    for vx0 in range(vx0_min, vx0_max + 1):
        n_min, n_max = step_bounds(vx0, target=target)
        vy0_min, vy0_max = velocity_bounds_y(n_min, n_max, target)
        for vy0 in range(vy0_min, vy0_max + 1):
            p,v,hit = run_trajectory((0,0), (vx0, vy0), target)
            if hit:
                velocity_hit_set.add((vx0,vy0))
                ytop = vy0/2*(vy0 + 1)
                if ytop > ytop_max:
                    ytop_max = ytop
                    #print(f'New max y with vel ({vx0, vy0}):', ytop_max)
                    velocity_best = (vx0, vy0)
                    
    return ytop_max, velocity_best, velocity_hit_set

def dec17_part2(target):
    y_target_min = target[1][0]
    y_target_max = target[1][1]
    y_range = range(min(y_target_min, y_target_max), max(y_target_min, y_target_max) - np.sign(max(y_target_min, y_target_max)))
    vx0_min, vx0_max = velocity_bounds_x(target=target)
    ytop_max = 0
    velocity_best = [0, 0]
    velocity_hit_set = set()

    for vx0 in range(vx0_min, vx0_max + 1):
        n_min, n_max = step_bounds(vx0, target=target)
        n_range = range(n_min, n_max + 1)
        vy0_min, vy0_max = velocity_bounds_y(n_min, n_max, target)
        for vy0 in range(vy0_min, vy0_max + 1):
            for n in n_range:
                y = vy0*n - n*(n - 1)/2
                if y in y_range:

                    velocity_hit_set.add((vx0,vy0))
                    ytop = vy0/2*(vy0 + 1)
                    if ytop > ytop_max:
                        ytop_max = ytop
                        velocity_best = (vx0, vy0)
                    
    return ytop_max, velocity_best, velocity_hit_set

target_test = [(20,30), (-5, -10)]
target = [(265,287), (-58, -103)]
ytop_max, velocity_best, velocity_hit_count = dec17_part2(target = target_test)

print(f'Highest peak at {ytop_max} with velocity {velocity_best}')
print(f'A total of {len(velocity_hit_count)} velocity pairs will hit the target')

Highest peak at 45.0 with velocity (6, 9)
A total of 112 velocity pairs will hit the target


# December 21

In [64]:
def data_prep_21(data):
    data_prep = [int(player[-1]) for player in data]
    return data_prep

input_test = ['Player 1 starting position: 4','Player 2 starting position: 8']
input_21 = scrape_data(day=21)
data_prep_21(data=input_21)

[8, 4]

In [65]:
# Part 1
def move_player(pos, moves):
    p = (pos - 1 + moves) % 10 + 1
#     if p == 0:
#         new_pos = 10
#     else:
#         new_pos = p
    new_pos = p
    return new_pos

def die_score(value_current):
    die_max = 100
    score = 3*value_current + 1 + 2
    value_new = value_current + 3
    return score, value_new

def player_turn(player_pos, die_pos):
    score, die_current = die_score(value_current=die_pos)
    player_pos_new = move_player(pos=player_pos, moves=score)
    
    return player_pos_new, die_current

def dec21_part1(data):
    player_positions_start = data_prep_21(data)
    pos_player_1 = player_positions_start[0]
    pos_player_2 = player_positions_start[1]
    
    die_current = 1
    score_player_1 = 0
    score_player_2 = 0
    die_rolls = 0
    
    while True:
        pos_player_1, die_current = player_turn(player_pos=pos_player_1, die_pos=die_current)
        score_player_1 += pos_player_1
        die_rolls += 3
        if (score_player_1 >= 1000):
            break
            
        pos_player_2, die_current = player_turn(player_pos=pos_player_2, die_pos=die_current)
        score_player_2 += pos_player_2
        die_rolls += 3
        if (score_player_2 >= 1000):
            break

    loser_score = min(score_player_1, score_player_2)
    winner_score = max(score_player_1, score_player_2)

    return loser_score, winner_score, die_rolls
    
loser_score, winner_score, die_rolls = dec21_part1(input_21)
print(loser_score*die_rolls)

504972


# December 25

In [66]:
# December 25
input_test = ['v...>>.vv>',
              '.vv>>.vv..',
                '>>.>v>...v',
                '>>v>>.>.v.',
                'v>v.vv.v..',
                '>.>>..v...',
                '.vv..>.>v.',
                'v.v..>>v.v',
                '....v..v.>']
input_test = np.array([list(row) for row in input_test])
input_25 = scrape_data(day=25)
input_25 = np.array([list(row) for row in input_25])
print('Input length:', len(input_25))
print('Test input:\n', np.array(input_test))

Input length: 137
Test input:
 [['v' '.' '.' '.' '>' '>' '.' 'v' 'v' '>']
 ['.' 'v' 'v' '>' '>' '.' 'v' 'v' '.' '.']
 ['>' '>' '.' '>' 'v' '>' '.' '.' '.' 'v']
 ['>' '>' 'v' '>' '>' '.' '>' '.' 'v' '.']
 ['v' '>' 'v' '.' 'v' 'v' '.' 'v' '.' '.']
 ['>' '.' '>' '>' '.' '.' 'v' '.' '.' '.']
 ['.' 'v' 'v' '.' '.' '>' '.' '>' 'v' '.']
 ['v' '.' 'v' '.' '.' '>' '>' 'v' '.' 'v']
 ['.' '.' '.' '.' 'v' '.' '.' 'v' '.' '>']]


In [67]:
def move_right(arr, symbol):
    arr_exp = np.concatenate((arr, arr[:,0].reshape(-1,1)), axis=1)
    arr_exp = np.insert(arr_exp, 0, arr[:,-1], axis=1)
    
    arr_symbol = (arr_exp == symbol)
    arr_occupied = (arr_exp == '>') | (arr_exp == 'v')
    
    move_right = (arr_symbol[:, :-1]^arr_occupied[:, 1:])*arr_symbol[:, :-1]
    move_cnt = np.sum(move_right)
    
    new_arr = arr.copy()
    new_arr[move_right[:, 1:]]= '.'

    new_arr[move_right[:, :-1]]= symbol

    return new_arr, move_cnt

def move_down(arr, symbol):
    arr_new, cnt = move_right(arr.T, symbol=symbol)
    
    return arr_new.T, cnt 

def dec_25_part1(data):
    steps = 0
    while True:
        steps += 1
        data, cnt_right = move_right(data, symbol='>')
        data, cnt_down = move_down(data, symbol='v')
        
        if cnt_right + cnt_down == 0:
            break
            
    return steps

steps = dec_25_part1(data=input_25)
print('Answer: No movement at step:', steps)

Answer: No movement at step: 432
